# 2. FastText demo

### About this notebook

This notebook was used in the 50.039 Deep Learning course at the Singapore University of Technology and Design.

**Author:** Matthieu DE MARI (matthieu_demari@sutd.edu.sg)

**Version:** 1.2 (29/08/2023)

This notebook demonstrates how you may reuse a pre-trained language model from a Python library (e.g. the fasttext one).

Most language models already have pre-trained version online, along with a few basic method giving the closest 10 words to a given word or vector representation, giving analogies, etc.

This is based on the paper P. Bojanowski*, E. Grave*, A. Joulin, T. Mikolov, "Enriching Word Vectors with Subword Information", 2017 (https://arxiv.org/abs/1607.04606).

And it follows the (very nice) documentation provided on https://fasttext.cc/docs/en/unsupervised-tutorial.html

**Requirements:**
- Python 3 (tested on v3.11.4)
- Matplotlib (tested on v3.7.2)
- Numpy (tested on v1.25.2)
- Torch (tested on v2.0.1+cu118)
- Fasttext (tested on v0.9.2)
- We also strongly recommend setting up CUDA on your machine!

**Important:** You might have to pip install the **fasttext** package.

Note: As of September 2023, the fasttext package seems to have a problem, will check for fix later in 2024.

### Imports

In [ ]:
import numpy as np
from numpy.linalg import norm
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import functools
import matplotlib.pyplot as plt
import fasttext
import fasttext.util
CUDA = torch.cuda.is_available()
device = torch.device("cuda" if CUDA else "cpu")

### Download the model

This command will download a pre-trained english language model and save it to file.

Note: heavy model, takes a while.

In [ ]:
# Load model
lang = 'en'
fasttext.util.download_model(lang, if_exists = 'ignore')  # English
model = fasttext.load_model('cc.en.300.bin')

### Getting a vector embedding for word

The command below can be used to get the word embedding for any word.

In [ ]:
# Get vector embedding for word
word = "hello"
v = model.get_word_vector(word)
print(v)

### Getting the closest words to a given word or vector

You may use the get_nearest_neighbors() method to get the 10 closest word in vocabulary to another given word. This function simply checks all the words in the vocabulary used to train the model (which you can see by asking Python to print model.words)

The function get_nearest_neighbors() shows the top 10 words, along with their cosine similarity score (with value between -1 and 1). The queried word is ignored as it as a maximal score of 1.

In [ ]:
# Show closest words and their similarity scores for a given word
l = model.get_nearest_neighbors('university')
print(l)

As mentionned earlier, in order to find nearest neighbors, we need to compute a similarity score between words. In our case, we use the cosine similarity between word vectors (to be discussed in W9S3), which can be simply implemented as the function below.

In [ ]:
# A simple cosine similarity function
def cos_sim(A, B):
    return np.dot(A, B)/(norm(A)*norm(B))

Let us now assume we have the following vector *vec*, as shown below, being produced by an AI as the result of the next word to put at the end of a sentence given as input (a typical autocomplete AI). Which word does it correspond to?

In [ ]:
# Let us assume we have the following vector, produced by an AI as the result of
# the next word to put at the end of a sentence given as input.
# Which word does it correspond to?
vec = np.array([2.43494716e-02, -1.13698803e-02, -8.15532170e-03, 5.03604002e-02, 1.25942141e-01, -2.88365828e-03, \ 
-2.06868462e-02, -4.69126739e-03, 4.58838940e-02, 1.73562765e-02, -8.25469662e-03, \ 
5.65736219e-02, -1.14337327e-02, 4.04844200e-03, 5.42512201e-02, 2.26573348e-02, \ 
-2.54051853e-03, 2.01502889e-02, -1.32389991e-02, 8.96624662e-03, 3.41498256e-02, \ 
7.60558695e-02, -1.27295889e-02, -8.14982690e-03, -5.47902379e-03, -1.07149500e-03, \ 
3.81412208e-02, 2.73656528e-02, -4.36594225e-02, 1.29438847e-01, -4.90466543e-02, \ 
4.29646932e-02, 2.86309374e-03, 5.30344844e-02, -1.41575374e-02, -3.32986973e-02, \ 
-2.85066571e-03, 3.23570818e-02, -2.29572412e-02, -6.47579320e-03, 6.47428632e-03, \ 
6.89802226e-03, 3.28526311e-02, 2.34083682e-02, -7.03127980e-02, 6.32729940e-03, \ 
2.51552891e-02, 3.32372226e-02, 2.74539664e-02, 1.01511143e-02, -1.13459498e-01, \ 
-2.98698284e-02, -2.86431052e-02, -3.44801601e-03, 7.84618855e-02, -3.26862857e-02, \ 
3.01900581e-02, -2.22665654e-03, -3.80350538e-02, -2.05339976e-02, 1.04580382e-02, \ 
7.08390912e-03, 4.72016744e-02, 1.05785560e-02, -2.41747163e-02, -1.91038251e-02, \ 
-2.29708627e-02, 6.75713178e-03, -3.81427929e-02, -1.19372606e-02, 3.85839911e-03, \ 
-4.21822034e-02, -3.34282927e-02, -5.04225977e-02, 5.95574416e-02, 6.21405877e-02, \ 
5.89391328e-02, 3.00594103e-02, 7.03570433e-03, -2.65786666e-02, -7.48579130e-02, \ 
-9.85325202e-02, -6.87865214e-03, -4.88833338e-02, -2.76888218e-02, -9.93961766e-02, \ 
-1.38292639e-02, 2.11257339e-02, 1.30583998e-03, -9.15561896e-03, -5.50810713e-03, \ 
1.36922840e-02, -3.30126770e-02, -1.24529023e-02, 3.46214436e-02, 2.10482087e-02, \ 
4.74231467e-02, 2.49306355e-02, 3.99287716e-02, -2.70848423e-02, 1.84357129e-02, \ 
-3.00431363e-02, -5.37705421e-02, 8.96717422e-04, -2.11293567e-02, 7.32455999e-02, \ 
1.89772695e-02, -2.76051089e-02, -5.50191998e-02, -5.64891510e-02, 1.90830119e-02, \ 
1.63993519e-02, 1.16397105e-02, 1.72323566e-02, -4.67531644e-02, -7.23658502e-03, \ 
-5.11930510e-02, 1.90194603e-02, -4.87585589e-02, -2.63743661e-03, 4.64431569e-03, \ 
2.68591009e-02, -9.08980370e-02, 4.56268936e-02, 9.81546566e-03, 4.43068780e-02, \ 
-5.09747863e-03, 6.53168112e-02, -5.50937243e-02, -4.40559387e-02, 7.82926753e-02, \ 
6.18038280e-03, 2.95869913e-03, -7.18970736e-03, -1.50787802e-02, -2.60494314e-02, \ 
5.51519282e-02, -2.91540623e-02, 1.19727165e-01, -4.89316583e-02, -3.47498730e-02, \ 
-7.97853172e-02, 1.05130011e-02, -1.10298898e-02, -5.67120053e-02, -5.37064895e-02, \ 
-6.42676502e-02, 4.72203782e-03, 4.20049019e-02, 7.56018907e-02, -2.58790590e-02, \ 
2.69915871e-02, 1.58740804e-02, -6.09181114e-02, -1.42964739e-02, -2.44542994e-02, \ 
4.45270538e-02, -1.94135867e-02, 1.47110242e-02, 2.05157958e-02, 1.59903448e-02, \ 
3.56206931e-02, 4.24724966e-02, 6.63566682e-03, -6.66409880e-02, -6.79737553e-02, \ 
-1.12438202e-02, -3.43243890e-02, 1.88426562e-02, 1.45432474e-02, -5.18854037e-02, \ 
-3.71178724e-02, -3.93002667e-02, -4.30256240e-02, 2.02595517e-02, -2.63931137e-03, \ 
-7.53693581e-02, 1.48520917e-02, 6.63546994e-02, -2.93932091e-02, -2.24413350e-02, \ 
-1.27807586e-02, 1.27461310e-02, -2.63963565e-02, 1.86937153e-02, 7.23727047e-03, \ 
4.97347452e-02, -5.82952835e-02, -3.43466178e-02, -2.12498493e-02, -6.71594590e-02, \ 
2.23266073e-02, -3.39246681e-03, -2.03753486e-02, -2.81394124e-02, -5.83796129e-02, \ 
2.77499929e-02, -7.03376457e-02, -2.01767348e-02, 2.54131258e-02, 6.72632009e-02, \ 
1.26513271e-02, 3.66020715e-03, -5.22914082e-02, -1.24335103e-02, 1.18855387e-04, \ 
1.89846344e-02, 5.36294002e-03, -9.15854611e-03, 6.92975745e-02, -4.52709198e-02, \ 
-6.41203811e-03, 9.00226831e-03, -6.07688203e-02, -1.06704377e-01, 2.52603628e-02, \ 
-7.30112642e-02, -7.25407675e-02, -2.77754515e-02, 1.14866411e-02, 2.33099200e-02, \ 
2.51908768e-02, 4.68313098e-02, 3.77839096e-02, -4.11637686e-02, -3.06773037e-02, \ 
3.96903865e-02, 4.61493768e-02, -5.65565005e-02, -3.03707458e-02, 3.71143073e-02, \ 
-2.64637452e-03, 1.50120147e-02, 5.96747994e-02, 2.64049694e-02, 2.34242976e-02, \ 
-9.20007937e-03, -3.06213629e-02, 6.02588207e-02, -1.69080645e-02, 7.93042332e-02, \ 
3.95114999e-03, -2.90442072e-02, -3.37193161e-02, 4.88179773e-02, -1.46116344e-02, \ 
1.17246881e-02, -3.56169138e-03, -1.25362445e-02, 4.70493771e-02, 2.88301706e-02, \ 
-1.84939429e-03, 5.66348340e-03, -2.50344798e-02, -2.97428854e-02, 2.61730123e-02, \ 
3.92851094e-03, 1.68573745e-02, 3.45252752e-02, 2.14923769e-02, 8.96413028e-02, \ 
-7.67029449e-03, 3.42127867e-02, 6.36689365e-03, -2.52138451e-02, 9.56472009e-03, \ 
6.42596930e-02, 3.54354084e-02, 2.25691590e-03, -4.17026319e-03, -5.41674718e-02, \ 
1.69327669e-03, -4.42275107e-02, -1.29532749e-02, 9.76711325e-03, 4.98655774e-02, \ 
-2.94282883e-02, -1.42690735e-02, -2.31541265e-02, -2.38652471e-02, 7.96326473e-02, \ 
-2.12089568e-02, -2.29584128e-02, -7.06688315e-02, 7.19109625e-02, -1.89979356e-02, \ 
-3.96940708e-02, -6.12189695e-02, 2.67267507e-02, 8.46604817e-03, -2.80338842e-02, \ 
-6.76728263e-02, -1.99874751e-02, -3.97956595e-02, -3.06904335e-02, 3.49237956e-02, \ 
-1.08037302e-02, 9.83848721e-02, 3.61668691e-02, 8.09734687e-03])

In that scenario, we can simply check all words in the vocabulary, one at a time, get their word embeddings (in *vec2*) and compute a cosine similarity between *vec* and *vec2*.

We can then simply keep track of the word with the highest cosine similarity score. As expected, the word is "asparagus" (*vec*, in the cell above, was the result of the operation *model.get_nearest_neighbors('university')*)

In [ ]:
# Search for the closest word, i.e. the one with the highest cosine 
# similarity scores with our given word vector
max_val = 0
best_match = model.words[0]
for index, word in enumerate(model.words):
    vec2 = model.get_word_vector(word)
    val = cos_sim(vec, vec2)
    if(val > max_val):
        max_val = val
        best_match = word
    if(index % 50000 == 0):
        pct = round(index/2000000*100, 1)
        print("- Progress {}/{} [{}%]".format(index, 2000000, pct)
print("Studying vector: ", vec)
print("Best match is: ", best_match)
print("Cosine similarity with best match: ", max_val)

### Word analogies

You may even use word analogies, e.g.: Following the analogy between Paris and France, what are the top 10 words having the same analogy with Italy? 

In [ ]:
# Play with analogies
# Following the analogy between Paris and France, which words have the same analogy with Italy?
l3 = model.get_analogies("paris", "france", "italy")
print(l3)